In [ ]:
import pandas as pd

folder = 'experiments/neg_dev_matched'

binary_finetune_pos_preds = pd.read_csv(folder + '/binary_finetune_pos_preds.tsv', header=None, delimiter='\t')
binary_finetune_pos_preds=binary_finetune_pos_preds.rename({0: "pos"}, axis=1)

binary_finetune_neg_preds = pd.read_csv(folder + '/binary_finetune_neg_preds.tsv', header=None, delimiter='\t')
binary_finetune_neg_preds=binary_finetune_neg_preds.rename({0: "neg"}, axis=1)

full_finetune_pos_preds = pd.read_csv(folder + '/full_two_way_finetune_pos_preds.tsv', header=None, delimiter='\t')
full_finetune_pos_preds=full_finetune_pos_preds.rename({0: "pos"}, axis=1)

full_finetune_neg_preds = pd.read_csv(folder + '/full_two_way_finetune_neg_preds.tsv', header=None, delimiter='\t')
full_finetune_neg_preds=full_finetune_neg_preds.rename({0: "neg"}, axis=1)

examples = pd.read_csv('neg_dev_matched.tsv', delimiter='\t')
binary_preds = pd.concat([binary_finetune_pos_preds, binary_finetune_neg_preds, examples], axis=1)
binary_preds['pos'] = binary_preds.apply(lambda row: 'c' if row['pos'] == 0 else 'e', axis=1)
binary_preds['neg'] = binary_preds.apply(lambda row: 'c' if row['neg'] == 0 else 'e', axis=1)
binary_preds['change'] = binary_preds.apply(lambda row: 1 if row['pos'] != row['neg'] else 0, axis=1)

full_preds = pd.concat([full_finetune_pos_preds, full_finetune_neg_preds, examples], axis=1)
full_preds['pos'] = full_preds.apply(lambda row: 'n' if row['pos'] == 0 else 'e', axis=1)
full_preds['neg'] = full_preds.apply(lambda row: 'n' if row['neg'] == 0 else 'e', axis=1)
full_preds['change'] = full_preds.apply(lambda row: 1 if row['pos'] != row['neg'] else 0, axis=1)

def combine_preds(row):
    return row['pos']+row['neg']

binary_preds['change_type'] = binary_preds.apply(combine_preds, axis=1)
full_preds['change_type'] = full_preds.apply(combine_preds, axis=1)


# display(binary_preds['change_type'].value_counts())
# display(full_preds['change_type'].value_counts())

print("percentage of initial examples: ")
print("binary: ")
total = len(binary_preds)
print(binary_preds['pos'].value_counts()/total)

print("full: ")
total = len(full_preds)
print(full_preds['pos'].value_counts()/total)


subset = full_preds[full_preds['pos']=="e"]
display(subset['neg'].value_counts()/len(subset))


In [ ]:
#first: pos logits
#second: neg logits
#after that: direct, indirect effects for each i

hidden_dim = 768
column_names = [None]*(2*hidden_dim+2)
column_names[0] = "pos_logits"
column_names[1] = "neg_logits"
for i in range(hidden_dim):
    column_names[i*2+2] = str(i) + "_dir"
    column_names[i*2+3] = str(i) + "_indir"

logits = pd.read_csv('experiments/may7/binary_finetune.tsv', header=None, delimiter='\t', names=column_names)

In [ ]:
entailment_logits = logits.copy()

In [ ]:
for column in logits.columns:
    entailment_logits[column] = logits[column].apply(lambda x: float(x.split(",")[0][1:]))

In [ ]:
display(entailment_logits)

In [ ]:
print(len(entailment_logits))

In [ ]:
#get average change for direct effects



#get average change for indirect effects

